# 단어의 표현 (Word Representation)


기계는 문자를 그대로 인식할 수 없기때문에 숫자로 변환



# 1 원-핫 인코딩 (One-Hot Encoding)

## 1.1 직접 구현해보기

### "원숭이, 바나나, 사과" 로 원-핫 인코딩을 한다면

In [2]:
# 인코딩 대상 단어들을 담은 리스트
word_ls = ['원숭이','바나나','사과']

In [13]:
# 원핫 인코딩 생성
#1. 단어(토큰)의 순서를 정해준다.
word2idx={}
{
    '원숭이' : 0,
    '바나나' : 1,
    '사과' : 2
}
for token in word_ls:
    if token not in word2idx.keys():
        word2idx[token] = len(word2idx)
    else:
        pass

In [14]:
word_ls

['원숭이', '바나나', '사과']

In [5]:
#2. 단어별 벡터 생성
vector_lst=[]
for token in word2idx.values():
    tem_vec=[0] * len(word2idx)
    tem_vec[token] = 1
    vector_lst.append(tem_vec)
vector_lst

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]

In [6]:
import numpy as np
np.array(vector_lst)

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [11]:
def one_hot_encode(tokens):
    word2idx={}
    for token in tokens:
        if token not in word2idx.keys():
            word2idx[token] = len(word2idx)
        else:
            pass
    vector_lst=[]
    for token in word2idx.values():
        tem_vec=[0] * len(word2idx)
        tem_vec[token] = 1
        vector_lst.append(tem_vec)
    return np.array(vector_lst)

In [12]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1, 0, 0],
       [0, 1, 0],
       [0, 0, 1]])

### "코끼리"라는 단어가 추가된다면?

In [15]:
word_ls = ['원숭이','바나나','사과','코끼리']

In [16]:
one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [ ]:
# 라벨인코딩 ?

## 1.3 sklearn 활용

- LabelEncoder() : 유니크한 범주형 값들을 1부터 증가하는 숫자로 인코딩
- OneHotEncoder() : 유니크한 범주형 값들을 n:1 행렬로 변환

함수명 | 설명
--|--
fit(X[, y])	| Fit OneHotEncoder to X.
fit_transform(X[, y])	| Fit OneHotEncoder to X, then transform X.
inverse_transform(X)	| Convert the back data to the original representation.
transform(X)	| Transform X using one-hot encoding.

In [17]:
# sklearn을 활용한 one-hot encoding
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# 예제 데이터 배열
values = array(word_ls)
print(values)

# 문자열에 숫자를 붙임
label_enc = LabelEncoder()
int_enc = label_enc.fit_transform(values)
print(int_enc)

# binary encode
onehot_enc = OneHotEncoder(sparse=False)
int_enc = int_enc.reshape(len(int_enc), 1) # n:1 matrix로 변환
print(int_enc)
onehot_enc = onehot_enc.fit_transform(int_enc)
print(onehot_enc)

# one-hot encoding 의 첫번째 배열을 값을 역으로 산출
inverted = label_enc.inverse_transform([argmax(onehot_enc[0, :])])
print(inverted)

['원숭이' '바나나' '사과' '코끼리']
[2 0 1 3]
[[2]
 [0]
 [1]
 [3]]
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
['원숭이']




---



# 2 밀집 벡터 (Dense Vector)

## 2-1 유사도 계산

### 2.1.1 유클리디언 거리(Euclidean distance)
두 벡터사이의 직선 거리. 피타고라스 정리를 생각하면 이해하기 쉬움

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/5/55/Euclidean_distance_2d.svg/220px-Euclidean_distance_2d.svg.png"  width="200"/>

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/795b967db2917cdde7c2da2d1ee327eb673276c0" width="350"/>

https://en.wikipedia.org/wiki/Euclidean_distance

In [18]:
word_embedding_dic = {
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

In [21]:
import numpy as np
# 유클리디언 거리 계산
def euclidean_dist(a, b):
    a, b=np.array(a), np.array(b)
    return np.sqrt(np.sum((a-b)**2))

# 사과와 바나나의 유클리디언 거리
euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나'])

0.7071067811865475

### 2.1.2 자카드 유사도(Jaccard index)

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/1f/Intersection_of_sets_A_and_B.svg/200px-Intersection_of_sets_A_and_B.svg.png" />

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/eaef5aa86949f49e7dc6b9c8c3dd8b233332c9e7" />

https://en.wikipedia.org/wiki/Jaccard_index

In [22]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

# 토큰화를 수행합니다.
token_s1 = s1.split()
token_s2 = s2.split()

# 자카드 유사도 계산


### 2.1.3 코사인 유사도(Cosine Similarity)

*  두 벡터간의 유사도를 측정하는 방법 중 하나
*  두 벡터 사이의 코사인을 측정
*  0도 = 1, 90도 = 0, 180도 = -1   ==> 1에 가까울수록 유사도가 높음




<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/1d94e5903f7936d3c131e040ef2c51b473dd071d" width='350'/>

https://en.wikipedia.org/wiki/Cosine_similarity

In [ ]:
# 코사인 유사도 계산


In [ ]:
# 사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))

In [ ]:

# 사과와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['원숭이']))

In [ ]:
# 바나나와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))



---



# 3 TF-IDF를 활용한 단어 벡터

### 3.1 직접 구현하기

weighting schema|weight|설명
--|--|--
term frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/91699003abf4fe8bdf861bbce08e73e71acf5fd4" />|=토큰빈도/문서내토큰빈도
inverse document frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/864fcfdc0c16344c11509f724f1aa7081cf9f657" />|=log(총문서갯수/(토큰이 등장한 문서수))

In [23]:
d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"

In [24]:
corpus = [d1, d2]

In [26]:
token_d1 = d1.split()
token_d2 = d2.split()

In [28]:
word2idx ={}
for token in token_d1+token_d2:
    if token not in word2idx.keys():
        word2idx[token] = len(word2idx)
    else:
        pass


In [31]:
tdm_lst = []
for tokens in [token_d1, token_d2]:
    temp_tdm=[0] * len(word2idx)
    for counting_token in word2idx:
        temp_tdm[word2idx[counting_token]] = tokens.count(counting_token)
    tdm_lst.append(temp_tdm)

In [32]:
tdm_lst

[[1, 2, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0], [1, 0, 1, 1, 1, 0, 1, 0, 1, 2, 1, 1]]

In [33]:
sum(tdm_lst[0])

10

In [37]:
tf_lst = [tdm_lst[0][i]/sum(tdm_lst[0]) for i in range(len(tdm_lst[0]))]
tf_lst2= [tdm_lst[1][i]/sum(tdm_lst[1]) for i in range(len(tdm_lst[1]))]

In [44]:
tf_lst_d1=[tdm_lst[0], tf_lst]
tf_lst_d2=[tdm_lst[1], tf_lst]

In [45]:
import pandas as pd
tf1_df = pd.DataFrame(tf_lst_d1, index=['d1','tf_d1'], columns=word2idx).T
tf2_df = pd.DataFrame(tf_lst_d2, index=['d2','tf_d2'], columns=word2idx).T

In [46]:
tf1_df

,d1,tf_d1
The,1.0,0.1
cat,2.0,0.2
sat,1.0,0.1
on,1.0,0.1
my,1.0,0.1
face,1.0,0.1
I,1.0,0.1
hate,1.0,0.1
a,1.0,0.1
dog,0.0,0.0


In [47]:
tf2_df

,d2,tf_d2
The,1.0,0.1
cat,0.0,0.2
sat,1.0,0.1
on,1.0,0.1
my,1.0,0.1
face,0.0,0.1
I,1.0,0.1
hate,0.0,0.1
a,1.0,0.1
dog,2.0,0.0


In [25]:
import numpy as np

# TF-IDF 계산


# document 내 token 등장비율 TF

# 단어 - 문서 등장 횟수 IDF

# 두 값의 곱


### 3.2 sklearn 활용

- feature_extraction.text.CountVectorizer() : 문장들에서 단어의 빈도수를 벡터화

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
count_vect = CountVectorizer()
countv = count_vect.fit_transform(corpus)
print(countv.toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(count_vect.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

- feature_extraction.text.TfidfVectorizer() : 문장들을 TF-IDF로 벡터화

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
tfidf_vect = TfidfVectorizer().fit(corpus)
tfidfv = tfidf_vect.transform(corpus)
print(tfidfv.toarray())
print(tfidf_vect.vocabulary_)

## 3.3 gensim 활용

- corpora.Dictionary - 토큰화한 문서들 내에서 index-단어 사전을 생성
- models.TfidfModel() - TF-IDF 벡터화

In [ ]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim import corpora

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]

doc_ls = [doc.split() for doc in corpus]
id2word = corpora.Dictionary(doc_ls)  # fit dictionary
corpus = [id2word.doc2bow(doc) for doc in doc_ls]  # convert corpus to BoW format

tfidf = TfidfModel(corpus)  # fit model
vector = tfidf[corpus[0]]  # apply model to the first corpus document

In [ ]:
tfidf[corpus][0]



---



# 4 LSA(Latent Semantic Analysis)를 활용한 단어 벡터

## 4.1 sklearn 활용

- decomposition.TruncatedSVD() : 특이값분해

In [48]:
doc_ls = [
    '바나나 사과 포도 포도',
    '사과 포도',
    '포도 바나나',
    '짜장면 짬뽕 탕수육',
    '볶음밥 탕수육',
    '짜장면 짬뽕',
    '라면 스시',
    '스시',
    '가츠동 스시 소바',
    '된장찌개 김치찌개 김치',
    '김치 된장',
    '비빔밥 김치'
]

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD

count_vect = CountVectorizer()
countv = count_vect.fit_transform(doc_ls)
svd = TruncatedSVD(n_components=2, algorithm='randomized', n_iter=100)
svd.fit(countv)

features = count_vect.get_feature_names_out() # 단어 집합. 1,000개의 단어가 저장됨.
for i in range(len(features)) :
    print("{} : {}".format(features[i], svd.components_[:,i]))

가츠동 : [2.94566118e-17 4.36962579e-16]
김치 : [-3.39399278e-19 -2.58393334e-15]
김치찌개 : [-1.08044177e-18 -1.27757205e-15]
된장 : [ 8.01344465e-18 -7.08924595e-16]
된장찌개 : [ 1.32359566e-18 -1.30327005e-15]
라면 : [-7.72719138e-18  1.56194319e-15]
바나나 : [ 4.08248290e-01 -1.58335038e-16]
볶음밥 : [-1.23042876e-22  1.27737006e-01]
비빔밥 : [-7.51683942e-18 -6.24085325e-16]
사과 : [4.08248290e-01 8.31549763e-17]
소바 : [2.35494873e-17 5.00652850e-16]
스시 : [ 6.59571930e-17 -1.81548747e-15]
짜장면 : [-5.89533838e-22  6.12024764e-01]
짬뽕 : [-5.89533838e-22  6.12024764e-01]
탕수육 : [-4.66490962e-22  4.84287758e-01]
포도 : [8.16496581e-01 3.75904405e-17]


## 4.2 gensim 활용

- models.LsiModel() : LSA 모델

In [50]:
doc_ls = [
    '바나나 사과 포도 포도',
    '사과 포도',
    '포도 바나나',
    '짜장면 짬뽕 탕수육',
    '볶음밥 탕수육',
    '짜장면 짬뽕',
    '라면 스시',
    '스시',
    '가츠동 스시 소바',
    '된장찌개 김치찌개 김치',
    '김치 된장',
    '비빔밥 김치'
]
doc_ls = [d.split() for d in doc_ls]

In [51]:
from gensim import corpora
from gensim.models import LsiModel

id2word = corpora.Dictionary(doc_ls) #사전 구축
corpus = [id2word.doc2bow(text) for text in doc_ls] # 코퍼스 생성
lsi = LsiModel(corpus, id2word=id2word, num_topics=2) #LSA 모델

for i in id2word.keys() :
    print("{} : {}".format(id2word[i], lsi.projection.u[i]))

바나나 : [4.08248290e-01 2.04413238e-16]
사과 : [ 4.08248290e-01 -2.03886206e-16]
포도 : [8.16496581e-01 3.38364459e-18]
짜장면 : [-1.70170841e-18  6.12024764e-01]
짬뽕 : [2.04855388e-17 6.12024764e-01]
탕수육 : [1.56208287e-17 4.84287758e-01]
볶음밥 : [-2.08050713e-18  1.27737006e-01]
라면 : [-3.08477492e-17 -9.29100828e-16]
스시 : [-1.94222392e-16 -2.84514324e-15]
가츠동 : [-3.50963269e-17 -1.27852964e-15]
소바 : [-5.09252188e-17 -1.21139830e-15]
김치 : [ 4.53371499e-17 -7.35987266e-15]
김치찌개 : [ 3.21131559e-17 -3.01517374e-15]
된장찌개 : [ 3.42108522e-17 -3.05284750e-15]
된장 : [ 5.0481303e-17 -2.1485403e-15]
비빔밥 : [ 1.94629086e-18 -2.36211607e-15]
